<a href="https://colab.research.google.com/github/gopalprashad81/HybridSeleniumFramework/blob/main/MachineLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================
# Core Libraries
# ============================
import numpy as np
import pandas as pd

# ============================
# Scikit-Learn (sklearn)
# ============================
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder

# ============================
# LightGBM
# ============================
import lightgbm as lgb

# ============================
# Visualization
# ============================
import matplotlib.pyplot as plt
import seaborn as sns

# ============================
# Optional: Suppress Warnings
# ============================
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# lecture 3
# rmse_train=metrics.root_mean_squared_error(y_true=train['y'],y_pred=old.predict(train.drop(columns
from sklearn.metrics import median_absolute_error
msdae_train = median_absolute_error(y_true=train['y'], y_pred=pls.predict(train.drop(columns=['y'])))
msdae_test =median_absolbute_error(x_test, x_pred_test)                                                                                     )))

In [4]:
import sklearn.linear_model

In [ ]:
huber=  HuberRegressor().fit(X_train,y_train)
y_pred_huber  = huber.predict(x_test)
# compare fits on test sample
rmse_ols =mean_squared_error(y_test,y_pred_ols)
rmse_huber = mean_sqaured_error(y_test,y_pred_huber,squared=False)

print("rmse_ols", rmse_ols)
print("rmse_huber",rmse_huber)